In [1]:
from PIL import Image

def traitement_image(file, texte):
    liste_binaire=[]
    im = Image.open(file)
    im = im.convert('RGBA')
    w_img=(im.size)[0]
    h_img=(im.size)[1]
    binaire=''.join(format(ord(i), '08b') for i in texte)#la variable binaire est un string de binaire correspondant au texte donne en argument
    for i in range(len(binaire)):
        liste_binaire.append(int(binaire[i])) #liste_binaire est une liste contenant des strings. chaque string est un nombre en binaire
    nb_pixels = len(liste_binaire)//4
    for k in range(nb_pixels):
        coordinate = x, y = k % w_img, k//w_img # on place les bits en partant du pixel en haut a droite. on incremente de un a un vers la gauche jusqua toucher la limite droite, on revient ensuite a gauche en descendant d'une ligne. donc l'abscisse est de k modulo la largeur de l'image, et l'ordonnee est la division entiere de k par la largeur de l'image.
        tpl = im.getpixel(coordinate)
        couleur = list(tpl)
        couleur_bin = [int((bin(couleur[0]))[2:10]), int((bin(couleur[1]))[2:10]), int((bin(couleur[2]))[2:10]),int((bin(couleur[3]))[2:10])] #couleur_bin est une liste. les valeurs prises sont 'couleur' en binaire, string, auquel on enleve le '0b' du debut(avec un slice), et qu'on reconvertit en string
        for i in range(4):
            if liste_binaire[4*k+i] != couleur_bin[i] % 2:
                couleur_bin[i]=str((couleur_bin[i]//10)*10+liste_binaire[4*k+i])
                couleur_bin[i]=int((couleur_bin)[i],2)
            else:
                couleur_bin[i]=int(str(couleur_bin[i]),2)
        couleur_bin=tuple(couleur_bin)
        im.putpixel(((k % w_img),(k//w_img)),couleur_bin)
    # la varibale liste_nb_pixels contient la valeur en binaire sur 32 bits du nombre de pixels a decoder
    liste_nb_pixels=list(bin(nb_pixels))
    liste_nb_pixels.pop(0)
    liste_nb_pixels.pop(0)
    liste_nb_pixels.reverse()
    for i in range(len(liste_nb_pixels)):
        liste_nb_pixels[i]=int(liste_nb_pixels[i])
    while len(liste_nb_pixels)<32:
        liste_nb_pixels.append(0)
    #je laisse la liste a l'envers car je vais prendre les pixels en partant du bit de poids faible, c'est plus simple pr la boucle
    for i in range(8):
        x=-1-i%w_img#j'utilise ces operations sur x et y pour que le nombre de pixel stocke soit ecrit meme si l'image fait moins de 8 pixels de large
        y=-1-i//w_img
        couleur_pixel=list(im.getpixel([x,y]))
        for k in range(4):
            if couleur_pixel[k]%2==1 and liste_nb_pixels[0]%2==0:
                couleur_pixel[k]-=1
            elif couleur_pixel[k]%2==0 and liste_nb_pixels[0]%2==1:
                couleur_pixel[k]+=1
            del(liste_nb_pixels[0])
        im.putpixel([x,y],tuple(couleur_pixel))
    im.show()
    im.save('pigeon_encode.png')

# programme principal
traitement_image('pigeon.png', 'hello')
#print(decodage('pigeon_encode.png'))
